<a href="https://colab.research.google.com/github/midnightripper/IDEC/blob/main/Copy_of_idec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import scipy.io
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from keras import backend as K
from keras.layers import Layer, InputSpec
def dataloader(path):
    data = scipy.io.loadmat(path)
    AF = data['AF']
    modified_rows1 = AF[:-2]
    last_rows1 = AF[-2:]
    CF = data['CF']
    concatenated_array = np.concatenate((modified_rows1, CF,last_rows1), axis=0)
    new=concatenated_array
    u, count = np.unique(new[-1], return_counts=True)
    a = u[np.logical_or(count < 2, count > 2)]
    c = new[:, np.isin(new[-1], a, invert=True)]
    x = c[0:-2]; y = c[-2]; w = c[-1];
    return x.T, y.T, w.T, data['CF_info']

def normalization(feats):
    df = pd.DataFrame(feats)
    scaler = StandardScaler()
    x_new = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    return x_new

In [3]:
import argparse
original_dim = 38

# train_path = filee; test_path = filee.replace('train','test')
train_path='/content/drive/MyDrive/finalData/TypicalFA_comb1/GER_train_fisher-2000_FA_GT_ESTphnTrans_estStress.mat'
test_path='/content/drive/MyDrive/finalData/TypicalFA_comb1/GER_test_fisher-2000_FA_GT_ESTphnTrans_estStress.mat'
x, y, wtrain1, info_train1 = dataloader(train_path);
xtest, ytest, wtest1, info_test1 = dataloader(test_path);
xtest= normalization(xtest)
x= normalization(x)

In [4]:
import numpy as np
from keras.layers import Dense, Input
from keras.models import Model
from keras.optimizers import SGD
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score
from scipy.optimize import linear_sum_assignment
import csv, os

def cluster_acc(y_true, y_pred):
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    _, col_ind = linear_sum_assignment(w.max() - w)
    return sum([w[i, j] for i, j in zip(range(w.shape[0]), col_ind)]) * 1.0 / y_pred.size

def autoencoder(dims, act='relu'):
    n_stacks = len(dims) - 1
    x = Input(shape=(dims[0],), name='input')
    h = x
    for i in range(n_stacks-1):
        h = Dense(dims[i + 1], activation=act, name='encoder_%d' % i)(h)
    h = Dense(dims[-1], name='encoder_%d' % (n_stacks - 1))(h)
    for i in range(n_stacks-1, 0, -1):
        h = Dense(dims[i], activation=act, name='decoder_%d' % i)(h)
    h = Dense(dims[0], name='decoder_0')(h)
    return Model(inputs=x, outputs=h)

In [5]:
class ClusteringLayer(Dense):
    def __init__(self, n_clusters, alpha=1.0, **kwargs):
        self.n_clusters = n_clusters
        self.alpha = alpha
        super(ClusteringLayer, self).__init__(n_clusters, **kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=tf.float32, shape=(None, input_dim))
        self.clusters = self.add_weight(shape=(self.n_clusters, input_dim),
                                        initializer='glorot_uniform', name='clusters')
        self.built = True

    def get_config(self):
        config = super(ClusteringLayer, self).get_config()
        config['n_clusters'] = self.n_clusters
        config['alpha'] = self.alpha
        return config

    def call(self, inputs):
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
        return q

In [6]:
import numpy as np
from keras.layers import Dense, Input
from keras.models import Model
from keras.optimizers import SGD
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score
from scipy.optimize import linear_sum_assignment
import csv, os

def cluster_acc(y_true, y_pred):
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    _, col_ind = linear_sum_assignment(w.max() - w)
    return sum([w[i, j] for i, j in zip(range(w.shape[0]), col_ind)]) * 1.0 / y_pred.size

def autoencoder(dims, act='relu'):
    n_stacks = len(dims) - 1
    x = Input(shape=(dims[0],), name='input')
    h = x
    for i in range(n_stacks-1):
        h = Dense(dims[i + 1], activation=act, name='encoder_%d' % i)(h)
    h = Dense(dims[-1], name='encoder_%d' % (n_stacks - 1))(h)
    for i in range(n_stacks-1, 0, -1):
        h = Dense(dims[i], activation=act, name='decoder_%d' % i)(h)
    h = Dense(dims[0], name='decoder_0')(h)
    return Model(inputs=x, outputs=h)

class ClusteringLayer(Dense):
    def __init__(self, n_clusters, alpha=1.0, **kwargs):
        self.n_clusters = n_clusters
        self.alpha = alpha
        super(ClusteringLayer, self).__init__(n_clusters, **kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=tf.float32, shape=(None, input_dim))
        self.clusters = self.add_weight(shape=(self.n_clusters, input_dim),
                                        initializer='glorot_uniform', name='clusters')
        self.built = True

    def get_config(self):
        config = super(ClusteringLayer, self).get_config()
        config['n_clusters'] = self.n_clusters
        config['alpha'] = self.alpha
        return config

    def call(self, inputs):
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
        return q

class IDEC(object):
    def __init__(self, dims, n_clusters=2, alpha=1.0):
        self.dims = dims
        self.input_dim = dims[0]
        self.n_stacks = len(self.dims) - 1
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.autoencoder = autoencoder(self.dims)

    def initialize_model(self, ae_weights=None, gamma=0.1, optimizer='adam'):
        if ae_weights is not None:
            self.autoencoder.load_weights(ae_weights)
            print('Pretrained AE weights are loaded successfully.')
        else:
            print('Training the autoencoder from scratch.')
            self.autoencoder.compile(loss='mse', optimizer=optimizer)
            self.autoencoder.fit(x, x, batch_size=self.batch_size, epochs=200)  # You can adjust the epochs as needed

        hidden = self.autoencoder.get_layer(name='encoder_%d' % (self.n_stacks - 1)).output
        self.encoder = Model(inputs=self.autoencoder.input, outputs=hidden)

        # prepare IDEC model
        clustering_layer = ClusteringLayer(self.n_clusters, name='clustering')(hidden)
        self.model = Model(inputs=self.autoencoder.input,
                           outputs=[clustering_layer, self.autoencoder.output])
        self.model.compile(loss={'clustering': 'kld', 'decoder_0': 'mse'},
                           loss_weights=[gamma, 1],
                           optimizer=optimizer)

    def fit(self, x, y=None, batch_size=256, maxiter=2e4, tol=1e-3, update_interval=140, save_dir='./results/idec'):
        print('Update interval', update_interval)
        save_interval = int(x.shape[0] / batch_size) * 5
        print('Save interval', save_interval)
        kmeans = KMeans(n_clusters=self.n_clusters, n_init=20)
        y_pred = kmeans.fit_predict(self.encoder.predict(x))
        y_pred_last = np.copy(y_pred)
        self.model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])

        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        logfile = open(save_dir + '/idec_log.csv', 'w')
        logwriter = csv.DictWriter(logfile, fieldnames=['iter', 'acc', 'nmi', 'ari', 'L'])
        logwriter.writeheader()

        loss = [0, 0, 0]
        index = 0
        for ite in range(int(maxiter)):
            if ite % update_interval == 0:
                q, _ = self.model.predict(x, verbose=0)
                p = self.target_distribution(q)
                y_pred = q.argmax(1)
                if y is not None:
                    acc = cluster_acc(y, y_pred)
                    nmi = normalized_mutual_info_score(y, y_pred)
                    ari = adjusted_rand_score(y, y_pred)
                    loss = np.round(loss, 5)
                    logwriter.writerow({'iter': ite, 'acc': acc, 'nmi': nmi, 'ari': ari, 'L': loss[0]})
                    print('Iter', ite, ': Acc', acc, ', nmi', nmi, ', ari', ari, '; loss=', loss)

                delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
                y_pred_last = np.copy(y_pred)

                if ite > 0 and delta_label < tol:
                    print('delta_label ', delta_label, '< tol ', tol)
                    print('Reached tolerance threshold. Stopping training.')
                    logfile.close()
                    break

            if (index + 1) * batch_size > x.shape[0]:
                loss = self.model.train_on_batch(x=x[index * batch_size::], y=[p[index * batch_size::], x[index * batch_size::]])
                index = 0
            else:
                loss = self.model.train_on_batch(x=x[index * batch_size:(index + 1) * batch_size],
                                                 y=[p[index * batch_size:(index + 1) * batch_size],
                                                    x[index * batch_size:(index + 1) * batch_size]])
                index += 1

            if ite % save_interval == 0:
                print('saving model to:', save_dir + '/IDEC_model_' + str(ite) + '.h5')
                self.model.save_weights(save_dir + '/IDEC_model_' + str(ite) + '.h5')

            ite += 1

        logfile.close()
        print('saving model to:', save_dir + '/IDEC_model_final.h5')
        self.model.save_weights(save_dir + '/IDEC_model_final.h5')

        return y_pred

    def predict_clusters(self, x):
        q, _ = self.model.predict(x, verbose=0)
        return q.argmax(axis=1)

    @staticmethod
    def target_distribution(q):
        weight = q ** 2 / q.sum(0)
        return (weight.T / weight.sum(1)).T

if __name__ == "__main__":
    # Implement the rest of your code here using the modified DEC and IDEC classes.

    # Example usage:

    # load dataset and preprocess (implement dataloader and normalization functions)
    x, y, wtrain1, info_train1 = dataloader(train_path)
    xtest, ytest, wtest1, info_test1 = dataloader(test_path)
    xtest = normalization(xtest)
    x = normalization(x)

    # Create and train the DEC model
    # dec = DEC(dims=[x.shape[-1], 500, 500, 2000, 10], n_clusters=2)
    # dec.pretrain(x, batch_size=256, epochs=200, optimizer='adam')
    # dec.compile(loss='kld', optimizer='adam')
    # dec.fit(x, y=y, batch_size=256, tol=0.001, maxiter=20000, update_interval=140, save_dir='./results/dec')
    # Load the autoencoder weights
    # Assuming you have already loaded the dataset `x` with shape (5000, 38) and labels `y`

    # Create and train the IDEC model
    idec = IDEC(dims=[x.shape[-1], 500, 500, 2000, 10], n_clusters=2)
    idec.initialize_model(ae_weights='autoencoder_weights.h5', gamma=0.1, optimizer='adam')
    idec.fit(x, y=y, batch_size=256, tol=0.001, maxiter=20000, update_interval=140, save_dir='./results/idec')

    # Show the final results
    y_pred = idec.predict_clusters(x)
    print('ACC:', cluster_acc(y, y_pred))


Pretrained AE weights are loaded successfully.
Update interval 140
Save interval 75
124/124 [==============================] - 1s 1ms/step
Iter 0 : Acc 0.5244702320887992 , nmi 0.001732987762655746 , ari 0.002143861061730856 ; loss= [0 0 0]
saving model to: ./results/idec/IDEC_model_0.h5
saving model to: ./results/idec/IDEC_model_75.h5
Iter 140 : Acc 0.5227043390514632 , nmi 0.0014893986780975656 , ari 0.0018102577153003472 ; loss= [0.0213  0.00527 0.02077]
saving model to: ./results/idec/IDEC_model_150.h5
saving model to: ./results/idec/IDEC_model_225.h5
Iter 280 : Acc 0.5216952573158425 , nmi 0.001362025781752033 , ari 0.0016312886030698712 ; loss= [0.02531 0.01188 0.02412]
saving model to: ./results/idec/IDEC_model_300.h5
saving model to: ./results/idec/IDEC_model_375.h5
Iter 420 : Acc 0.5229566094853683 , nmi 0.0015281564100230992 , ari 0.0018570060961422743 ; loss= [0.03061 0.0175  0.02886]
saving model to: ./results/idec/IDEC_model_450.h5
saving model to: ./results/idec/IDEC_mode

In [7]:
def calculate_accuracy(arr1, arr2):
    count = sum(1 for itr1, itr2 in zip(arr1, arr2) if itr1 == itr2)
    return count / len(arr1)

In [8]:
print(calculate_accuracy(y,y_pred))

0.5143794147325933


In [ ]:
def flip(arr):
  result = []
  for num in arr:
      if num == 1:
          result.append(0)
      else:
          result.append(1)
  return result

In [ ]:
y_pred_flip=flip(y_pred)

In [ ]:
print(calculate_accuracy(y,y_pred_flip))

0.4760343087790111


In [ ]:
print(y_pred)

[0 1 0 ... 1 1 1]
